In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [8]:
glyc_disease_message = one_hop_message('biolink:ChemicalSubstance','biolink:Disease',subject_id = "CHEBI:17754")
printjson(glyc_disease_message)
glyc_disease_id = submit_to_ars(glyc_disease_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:Disease"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=397b36b3-e179-44da-aff5-50ca2608dbf2


In [9]:
glyc_phen_message = one_hop_message('biolink:ChemicalSubstance','biolink:PhenotypicFeature',subject_id = "CHEBI:17754")
printjson(glyc_phen_message)
glyc_phen_id = submit_to_ars(glyc_phen_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:PhenotypicFeature"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=67685b9e-89a4-4ccf-8904-b95cc49c49ad


In [17]:
m = {"message": {
        "query_graph": {
  "nodes": {
    "n0": {
      "name": "Glycerol",
      "id": "PUBCHEM.COMPOUND:753"
    },
    "n1": {
      "category": "biolink:PhenotypicFeature"
    },
      
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}}}

In [18]:
glyc_phen_gkd_id = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=582d1596-5653-4d52-85d8-732269a07eb3


In [16]:
result = retrieve_ars_results(glyc_phen_gkd_id)

Error
Error ara-aragorn 0
Error ara-arax 0
Error ara-bte 0
Error ara-unsecret 0
Running kp-genetics 0
Running kp-molecular 0


# What can we do about reducing glycerol?

**Gene biolink:negatively_regulates_entity_to_entity Glycerol**

In [10]:
gene_nree_glyc_message = one_hop_message('biolink:Gene','biolink:ChemicalSubstance',predicate='biolink:negatively_regulates_entity_to_entity',object_id = "CHEBI:17754")
printjson(gene_nree_glyc_message)
gene_nree_glyc_id = submit_to_ars(gene_nree_glyc_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Gene"
                },
                "b": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:negatively_regulates_entity_to_entity"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=13fee52b-8b7b-4775-ad17-79a165cf56fb


**Gene biolink:negatively_regulates Glycerol**

In [11]:
gene_nr_glyc_message = one_hop_message('biolink:Gene','biolink:ChemicalSubstance',predicate='biolink:negatively_regulates',object_id = "CHEBI:17754")
printjson(gene_nr_glyc_message)
gene_nr_glyc_id = submit_to_ars(gene_nr_glyc_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Gene"
                },
                "b": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:negatively_regulates"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=c418c8d0-26ff-4826-a223-441892e3ff49


**Glycerol biolink:negatively_regulated_by Gene**

In [15]:
glyc_nrb_gene_message = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',predicate='biolink:entity_negatively_regulated_by_entity',subject_id = "CHEBI:17754")
printjson(glyc_nrb_gene_message)
glyc_nrb_gene_id = submit_to_ars(glyc_nrb_gene_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:Gene"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:entity_negatively_regulated_by_entity"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=07298522-6516-4cc8-82c7-f6c241b63be6


**Chemical biolink:negatively_regulates_entity_to_entity Glycerin**

In [13]:
chem_nree_glyc_message = one_hop_message('biolink:ChemicalSubstance','biolink:ChemicalSubstance',predicate='biolink:negatively_regulates_entity_to_entity',object_id = "CHEBI:17754")
printjson(chem_nree_glyc_message)
chem_nree_glyc_id = submit_to_ars(chem_nree_glyc_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:negatively_regulates_entity_to_entity"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=ff998f10-d3e5-49be-ae5c-aab894ada4b6


**Glycerin biolink:negatively_regulated_by Chemical**

In [14]:
glyc_nrb_chem_message = one_hop_message('biolink:ChemicalSubstance','biolink:ChemicalSubstance',predicate='biolink:negatively_regulated_by',subject_id = "CHEBI:17754")
printjson(glyc_nrb_chem_message)
glyc_nrb_chem_id = submit_to_ars(glyc_nrb_chem_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:negatively_regulated_by"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=172c78d4-2936-4568-bdad-bb3148d75f92


In [18]:
brett_q = {
  "message": {
    "query_graph": {
      "edges": {
        "e00": {
          "subject": "n00",
          "object": "n01"
        },
        "e01": {
          "subject": "n01",
          "object": "n02"
        },
        "e02": {
          "subject": "n02",
          "object": "n03"
        },
        "e03": {
          "subject": "n03",
          "object": "n04"
        },
        "e04": {
          "subject": "n04",
          "object": "n05"
        }
      },
      "nodes": {
        "n00": {
          "id": "DOID:0060363",
          "category": "biolink:Disease"
        },
        "n01": {
          "category": "biolink:Gene"
        },
        "n02": {
          "category": "biolink:Protein"
        },
        "n03": {
          "category": "biolink:MolecularActivity"
        },
        "n04": {
          "category": "biolink:MolecularActivity"
        },
        "n05": {
          "category": "biolink:ChemicalSubstance"
        }
      }
    }
  }
}

In [19]:
brett_id = submit_to_ars(brett_q)

https://arax.ncats.io/?source=ARS&id=93bb2f5d-1599-4a1e-b580-415921bd6b75
